In [6]:
import pandas as pd
import pandas_profiling
import src.datajanitor as datajanitor
import numpy as np
import seaborn
import matplotlib.pyplot as plt


shopperDs = datajanitor.getDataset('shoppers')
shopperDs.fetchData()
shoppers = pd.read_csv(shopperDs.fullFilePath)

In [7]:
shoppers.sample(5)
shoppers.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157213,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


In [ ]:
profile = pandas_profiling.ProfileReport(shoppers)
profile.to_file()

In [8]:
# Profile shows high corellation between `BounceRates` and `ExitRate`, so drop ExitRate
shoppers = shoppers.drop(labels=['ExitRates'], axis=1)

In [9]:
# Convert misclassified categorical variables back to categorical
shoppers['OperatingSystems'] = shoppers['OperatingSystems'].astype('category')
for col in ['Browser', 'Region', 'TrafficType', 'VisitorType', 'Month']:
    shoppers[col] = shoppers[col].astype('category')

# save a list of numeric (for scaling) columns and categorical columns
# adapted from https://stackoverflow.com/questions/29803093/check-which-columns-in-dataframe-are-categorical
numCols = shoppers.select_dtypes(include=['int64', 'float64']).columns
catCols = shoppers.select_dtypes(include=['bool', 'category']).columns

# convert boolean types to 0 or 1 uint8
shoppers.Weekend = shoppers.Weekend.astype('uint8')
shoppers.Revenue = shoppers.Revenue.astype('uint8')

print(shoppers['Revenue'])

print('Numeric columns: ', numCols)
print('Categorical columns: ', catCols)
shoppers.dtypes

0        0
1        0
2        0
3        0
4        0
        ..
12325    0
12326    0
12327    0
12328    0
12329    0
Name: Revenue, Length: 12330, dtype: uint8
Numeric columns:  Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'PageValues', 'SpecialDay'],
      dtype='object')
Categorical columns:  Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
       'VisitorType', 'Weekend', 'Revenue'],
      dtype='object')


Administrative                int64
Administrative_Duration     float64
Informational                 int64
Informational_Duration      float64
ProductRelated                int64
ProductRelated_Duration     float64
BounceRates                 float64
PageValues                  float64
SpecialDay                  float64
Month                      category
OperatingSystems           category
Browser                    category
Region                     category
TrafficType                category
VisitorType                category
Weekend                       uint8
Revenue                       uint8
dtype: object

In [ ]:
# Find missing values and deal with them -- None!
print(shoppers.isnull().sum())

In [10]:
# Encode categorical and boolean columns
# Based off https://towardsdatascience.com/encoding-categorical-features-21a2651a065c
categoricalCols = shoppers.dtypes == 'category'
shoppers_ohe = pd.get_dummies(shoppers)

# join into original dataset and drop categories
shoppers = pd.concat([shoppers, shoppers_ohe], axis=1)
shoppers.drop(catCols, axis=1, inplace=True)
shoppers.sample(5)

KeyError: 'Revenue'